In [26]:
import cellCnn
import importlib

importlib.reload(cellCnn)
import random
import numpy as np
### (from: https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/NK_cell_ungated.ipynb ) AND https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/PBMC.ipynb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import seaborn as sns
from matplotlib import pyplot as plt, patches

from cellCnn.utils import mkdir_p
from cellCnn.plotting import plot_results
from cellCnn.utils import mkdir_p
from cellCnn.utils import save_results
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.ms.utils.helpers import calc_frequencies

import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [27]:
#%reset

In [28]:
##### state vars
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
infile = 'cohort_denoised_clustered_diagnosis_patients.csv'
indir = 'data/input'
outdir = 'out_ms_default'
rand_seed = 123
train_perc = 0.7
test_perc = 0.3
batch_size_pheno = 840  # deprecated  # so a size of 8425 is about equally sized in batches
batch_size_cd4 = 550  # deprecated #so a size of 550 gets me 16 batches for cd4
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}

In [29]:
np.random.seed(rand_seed)
mkdir_p(outdir)
df = pd.read_csv(os.path.join(indir, infile), index_col=0)
df = df.drop_duplicates()  ### reduces overfitting at cost of fewer data
df.shape
##### no duplicates in

(16889, 38)

In [30]:
# abundancy values of cell-types:
# cluster
# 0  b   0.133519
# 1  cd4   0.521227
# 3  nkt   0.021671
# 8  cd8   0.235123
# 10  nk  0.046184
# 11 my   0.024750
# 16 dg  0.017526

## RRMS # cluster
# 0     0.131616
# 1     0.508270
# 3     0.017368
# 8     0.249646
# 10    0.047377
# 11    0.025520
# 16    0.020203

### NINDC # cluster
# 0     0.135430
# 1     0.534243
# 3     0.025994
# 8     0.220534
# 10    0.044985
# 11    0.023976
# 16    0.014837
### for a filter that selected all cells
grouped = df.groupby('cluster').count()
relative = grouped / df.shape[0]
relative.iloc[:, 0]

rrms_df = df[df['diagnosis'] == 'NINDC']
grouped = rrms_df.groupby('cluster').count()
relative = grouped / rrms_df.shape[0]
relative.iloc[:, 0]

cluster
0     0.135430
1     0.534243
3     0.025994
8     0.220534
10    0.044985
11    0.023976
16    0.014837
Name: CD69, dtype: float64

In [34]:
### amount Cells per cell type ?
grouped = df.groupby('cluster').count()
grouped.iloc[:, 0]


cluster
0     2255
1     8803
3      366
8     3971
10     780
11     418
16     296
Name: CD69, dtype: int64

In [7]:
plt.figure()
patients_clusters_conf_table = pd.crosstab(df['gate_source'], df['cluster'])
sns.heatmap(patients_clusters_conf_table, annot=False, vmax=100)
plt.show()
#plt.savefig('images/patient_vs_cluster_conf_table.png')
patients_clusters_conf_table

/var/folders/p1/5d38cf1d6tvdyny38zb_66540000gn/T/ipykernel_12504/3510062378.py:4: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


cluster,0,1,3,8,10,11,16
gate_source,,,,,,,
3.0,60,119,6,30,5,5,0
4.0,15,46,0,45,2,1,13
5.0,33,131,0,70,2,2,4
8.0,24,122,0,31,10,11,4
9.0,72,270,2,137,11,10,4
...,...,...,...,...,...,...,...
128.0,4,6,4,6,1,6,0
130.0,24,82,2,56,8,4,0
131.0,20,48,2,14,13,7,0


In [8]:
print('Mean abundancies')
print('b: ' + str(patients_clusters_conf_table.iloc[:, 0].mean()))
print('CD4: ' + str(patients_clusters_conf_table.iloc[:, 1].mean()))
print('NKT: ' + str(patients_clusters_conf_table.iloc[:, 2].mean()))
print('CD8: ' + str(patients_clusters_conf_table.iloc[:, 3].mean()))
print('NK: ' + str(patients_clusters_conf_table.iloc[:, 4].mean()))
print('My: ' + str(patients_clusters_conf_table.iloc[:, 5].mean()))
print('dg: ' + str(patients_clusters_conf_table.iloc[:, 6].mean()))


Mean abundancies
b: 36.37096774193548
CD4: 141.98387096774192
NKT: 5.903225806451613
CD8: 64.04838709677419
NK: 12.580645161290322
My: 6.741935483870968
dg: 4.774193548387097


In [9]:
# pitch: key = gate_source, value = dict{diagnosis: df OR freq?}
rrms_df = df[df['diagnosis'] == 'RRMS']
rrms_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                    rrms_df.groupby('gate_source')}
nindc_df = df[df['diagnosis'] == 'NINDC']
nindc_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                     nindc_df.groupby('gate_source')}

In [10]:
print('RRMS cell-type abundances')
rrms_df.groupby('cluster').count()
print('Mean abundancy / patient is 273,032258065')

RRMS cell-type abundances
Mean abundancy / patient is 273,032258065


In [11]:
print('NINDC cell-type abundances')
nindc_df.groupby('cluster').count()

NINDC cell-type abundances


,CD69,CXCR5,CCR2,CD57,CD28,IL.9,IL.4,CD27,IL.3,IL.13,...,CD25,CD4,CD3,IL.21,IL.17A,CXCR4,CCR4,CD14,diagnosis,gate_source
cluster,,,,,,,,,,,,,,,,,,,,,
0,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141,...,1141,1141,1141,1141,1141,1141,1141,1141,1141,1141
1,4501,4501,4501,4501,4501,4501,4501,4501,4501,4501,...,4501,4501,4501,4501,4501,4501,4501,4501,4501,4501
3,219,219,219,219,219,219,219,219,219,219,...,219,219,219,219,219,219,219,219,219,219
8,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858,...,1858,1858,1858,1858,1858,1858,1858,1858,1858,1858
10,379,379,379,379,379,379,379,379,379,379,...,379,379,379,379,379,379,379,379,379,379
11,202,202,202,202,202,202,202,202,202,202,...,202,202,202,202,202,202,202,202,202,202
16,125,125,125,125,125,125,125,125,125,125,...,125,125,125,125,125,125,125,125,125,125


In [12]:
importlib.reload(cellCnn.ms.utils.helpers)
from cellCnn.ms.utils.helpers import *

#### here we could see freq differences across the 2 groups
print('Frequencies: ')
rrms_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df in
                      rrms_patients2df.items()}
nindc_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df
                       in nindc_patients2df.items()}
print('DONE')
# we got 31 patients each

Frequencies: 


KeyboardInterrupt: 

In [17]:
#### To get true frequencies we need to get rid of 0 entries (there are patient without some cells type due to data)
print('RRMS')
rrms_freq_df = pd.DataFrame(list(rrms_patients_freq.values()), columns=list(cluster_to_celltype_dict.values()))
rrms_freq_df = rrms_freq_df.replace(0, np.NaN)  ## this lets us skip the 0 entries
rrms_freq_df.describe()

RRMS


,b,cd4,nkt,cd8,nk,my,dg
count,31.000000,31.000000,28.000000,31.000000,31.000000,31.000000,25.000000
mean,0.145822,0.470019,0.034702,0.242621,0.047500,0.038641,0.029826
std,0.062796,0.127876,0.043881,0.068406,0.031251,0.041190,0.026293
min,0.044118,0.222222,0.003953,0.120543,0.008264,0.004662,0.001832
25%,0.098750,0.393336,0.008248,0.193943,0.022284,0.017099,0.011655
50%,0.136364,0.461538,0.018236,0.243386,0.045455,0.023810,0.019802
75%,0.190358,0.543535,0.032926,0.283517,0.057529,0.044735,0.042857
max,0.285714,0.738540,0.178571,0.349057,0.125000,0.222222,0.128205


In [18]:
print('NINDC')
nindc_freq_df = pd.DataFrame(list(nindc_patients_freq.values()), columns=list(cluster_to_celltype_dict.values()))
nindc_freq_df = nindc_freq_df.replace(0, np.NaN)
nindc_freq_df.describe()

NINDC


,b,cd4,nkt,cd8,nk,my,dg
count,31.000000,31.000000,30.000000,31.000000,30.000000,29.000000,26.000000
mean,0.160800,0.440072,0.043986,0.245588,0.059403,0.039039,0.020230
std,0.107951,0.155058,0.038204,0.094339,0.047733,0.033544,0.020225
min,0.033149,0.189189,0.001524,0.046559,0.011236,0.001706,0.002841
25%,0.076150,0.329932,0.012810,0.206028,0.024907,0.014587,0.007189
50%,0.142857,0.390476,0.039086,0.247573,0.037789,0.031250,0.017101
75%,0.215665,0.516367,0.067342,0.290557,0.086959,0.050000,0.024923
max,0.433198,0.784572,0.142857,0.410061,0.200000,0.142857,0.106557


In [ ]:
# reassign cluster, then save as other name or so
selected_cells_mtl = pd.read_csv('class_cd8/selected_cells/cohort_selected_cells.csv', header=0)
selected_cells = pd.read_csv('class/selected_cells/cohort_selected_cells.csv', header=0)

In [ ]:
# filter them by row[1] (g_thrres comparison if they are over 0)
# then make a comparison of clusters between the stl and mtl for the individual filters
# maybe plot amount of cd8 cluster cells for each filter (in both) and then compare

In [15]:
from sklearn.manifold import TSNE

#### t-SNE plot OVERVIEW
cluster = df.loc[:, 'cluster'].astype(int).reset_index(drop=True)
cluster_to_color_dict = {0: u'orchid', 1: u'darkcyan', 3: u'grey', 8: u'dodgerblue', 10: u'red', 11: u'turquoise',
                         16: u'darkviolet'}
cluster_to_color_series = cluster.replace(cluster_to_color_dict, regex=True)

samples = [df.iloc[:, :len(cytokines)]]
sample_names = ['cohort']
x = samples[0].reset_index(drop=True)
x_for_tsne = x.iloc[np.random.choice(x.shape[0], 1000), :]
x_tsne = TSNE(n_components=2).fit_transform(x)
x_tsne_df = pd.DataFrame(x_tsne)

/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [24]:
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}

label_color = [cluster_to_color_dict[l] for l in cluster]

fig.clf()
fig = plt.figure(figsize=(15, 15))
plt.scatter(x_tsne_df.iloc[:, 0], x_tsne_df.iloc[:, 1], s=2.5, marker='o', c=label_color,
            alpha=1, edgecolors='face')
for category in cluster.unique():
    plt.annotate(cluster_to_celltype_dict[category], x_tsne_df.loc[cluster == category, [0, 1]].mean(), fontsize=22,
                 fontweight="bold", color='black',
                 bbox=dict(fc="white"))
plt.title(f'T-SNE MS-cohort')
plt.savefig('.'.join(['../thesis/t-SNE_MS_cohort', 'png']),
            format='png', dpi=600)

plt.clf()
plt.close()
print('done')

done
